# Disaster Tweets

Pierre Mulliez individual assignment 

Lets inspect the dataset:

## Summary 

**About this analysis** 
<br>
We would like to know if a tweet represents a disaster or not.
<br>
<br>
**Why this analysis**
<br>
Prevention can save lives, with a population connected to social media like never before tweets analysis can optimize the responce time of rescue services.
<br>
<br>
**How** 
<ul>
<li> Exploding the text of the original datasets</li>
<li> Cleaning and Standardizing the dataset along the way</li>
    <ul>
        <li> Duplicates values</li>
        <li> Entry errors</li>
    </ul>
<li> Creating and joigning other datasets </li>
    <ul>
        <li> Sentiment analysis from the data architecture class</li>
        <li> Built my own disaster dataset reference words</li>
    </ul>
<li> note: I decided not to use the target variable int he exploratory analysis as this would be cheating </li>
</ul>

<br>

**Insights** 
<br>
**Answering key business questions**

1. Do the persons need assistance ?
2. Gravity of the situation ?
3. Can we infer a state with a higher rate of incidents ?

In [2]:
#https://www.kaggle.com/c/nlp-getting-started/data
import pandas as pd

In [3]:
#importing APIs
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType
from pyspark.sql.window import Window

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [4]:
#defining dataset's schema 
disasterSchema = StructType(\
    [StructField("id",IntegerType(),True),\
     StructField("keyword",StringType(),True),\
     StructField("text",StringType(),True),\
     StructField("target",IntegerType(),True)])

In [5]:
disaster_with_schema = spark.read.load("data.csv",
                     format="csv", sep=",", Schema=disasterSchema, header="true", mode = "PERMISSIVE")
print(disaster_with_schema.show(3))
disaster_with_schema.select(count("id").alias("Total_tweets")).show()

#sampling the dataset
sample_disaster = disaster_with_schema.sample(fraction = 0.6, seed=1234)
sample_disaster.select(count("id").alias("Total__sample_tweets")).show()

+---+-------+--------+--------------------+------+
| id|keyword|location|                text|target|
+---+-------+--------+--------------------+------+
|  1|   null|    null|Our Deeds are the...|     1|
|  4|   null|    null|Forest fire near ...|     1|
|  5|   null|    null|All residents ask...|     1|
+---+-------+--------+--------------------+------+
only showing top 3 rows

None
+------------+
|Total_tweets|
+------------+
|        8387|
+------------+

+--------------------+
|Total__sample_tweets|
+--------------------+
|                5052|
+--------------------+



In [6]:
disaster_with_schemapd = disaster_with_schema.toPandas()
disaster_with_schemapd.head()

,id,keyword,location,text,target
0,1,None,None,Our Deeds are the Reason of this #earthquake M...,1
1,4,None,None,Forest fire near La Ronge Sask. Canada,1
2,5,None,None,All residents asked to 'shelter in place' are ...,1
3,6,None,None,"13,000 people receive #wildfires evacuation or...",1
4,7,None,None,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
#sentimentschema
sentimentSchema = StructType(\
    [StructField("intensivity",StringType(),True),\
     StructField("numb",IntegerType(),True),\
     StructField("word",StringType(),True),\
     StructField("type",StringType(),True),\
     StructField("ny",StringType(),True),\
     StructField("positivity",StringType(),True)])

In [8]:
#Import sentiment dictionnary 
sentiment_with_schema = spark.read.load("dataset/dictionary.tsv",
                     format="csv", sep=r'\t', Schema=sentimentSchema, header="false", mode = "PERMISSIVE")
 
sentiment_with_schema.show(3)
pdsentiment = sentiment_with_schema.toPandas()
pdsentiment[1:4]

+--------+---+-----------+----+---+--------+
|     _c0|_c1|        _c2| _c3|_c4|     _c5|
+--------+---+-----------+----+---+--------+
|weaksubj|  1|  abandoned| adj|  n|negative|
|weaksubj|  1|abandonment|noun|  n|negative|
|weaksubj|  1|    abandon|verb|  y|negative|
+--------+---+-----------+----+---+--------+
only showing top 3 rows



,_c0,_c1,_c2,_c3,_c4,_c5
1,weaksubj,1,abandonment,noun,n,negative
2,weaksubj,1,abandon,verb,y,negative
3,strongsubj,1,abase,verb,y,negative


In [9]:
#panda summary
pdsentiment = pdsentiment.rename(columns={"_c0":"intensity","_c2": "word","_c3":"word_type", "_c4":"ny", "_c5":"posititvity"})
print(pdsentiment[1:4])
print("")
print(pdsentiment.describe())

    intensity _c1         word word_type ny posititvity
1    weaksubj   1  abandonment      noun  n    negative
2    weaksubj   1      abandon      verb  y    negative
3  strongsubj   1        abase      verb  y    negative

         intensity   _c1  word word_type    ny posititvity
count         8221  8221  8221      8221  8221        8221
unique           2     1  6878        13     7           7
top     strongsubj     1     1       adj     n    negative
freq          5569  8221     8      3249  6595        4902


In [10]:
#Splitting the text in panda /summary 
disaster_with_schemapd["new"]= disaster_with_schemapd["text"].str.split(" ")
print(disaster_with_schemapd.head(5))
print("")
print(disaster_with_schemapd.describe())
#print(disaster_with_schemapd['new'][3][1]) #query single element list of list 
print("Class of new text:")
print(type(disaster_with_schemapd['new']))

  id keyword location                                               text  \
0  1    None     None  Our Deeds are the Reason of this #earthquake M...   
1  4    None     None             Forest fire near La Ronge Sask. Canada   
2  5    None     None  All residents asked to 'shelter in place' are ...   
3  6    None     None  13,000 people receive #wildfires evacuation or...   
4  7    None     None  Just got sent this photo from Ruby #Alaska as ...   

  target                                                new  
0      1  [Our, Deeds, are, the, Reason, of, this, #eart...  
1      1     [Forest, fire, near, La, Ronge, Sask., Canada]  
2      1  [All, residents, asked, to, 'shelter, in, plac...  
3      1  [13,000, people, receive, #wildfires, evacuati...  
4      1  [Just, got, sent, this, photo, from, Ruby, #Al...  

            id keyword location  \
count     8387    7987     5080   
unique    8302     223     3340   
top     #Truth       0      USA   
freq         7     246      10

In [11]:
#In pyspark 
disaster_sample_expl = sample_disaster.select("id","keyword","location", explode(split(col("text"), " ")).alias("word"),"target")

w = Window().orderBy("id")
#Add an ID for all word, set an hashtag column, get rid of the hashtag in the word column 
disaster_sample_expl = disaster_sample_expl.select(row_number().over(w).alias("mainID"), col("*"))\
                        .withColumn(("hashtag"),regexp_extract(col('word'), '#.*', 0))\
                        .withColumn(("word"),regexp_extract(col('word'), '[a-zA-Z].*', 0))

#a lower column is created for optimizing merging capacities with other tables (as seen later)
disaster_sample_expl = disaster_sample_expl.select("*",lower(col('word')).alias("lword"))
disaster_sample_expl.show(25)
disaster_sample_expl.select(count("id").alias("Total_words")).show() 
disaster_sample_expl.select(countDistinct("location").alias("locations")).show()

+------+-----+-------+------------------+----------+------+---------+----------+
|mainID|   id|keyword|          location|      word|target|  hashtag|     lword|
+------+-----+-------+------------------+----------+------+---------+----------+
|     1|   10|   null|              null|     flood|     1|   #flood|     flood|
|     2|   10|   null|              null|  disaster|     1|#disaster|  disaster|
|     3|   10|   null|              null|     Heavy|     1|         |     heavy|
|     4|   10|   null|              null|      rain|     1|         |      rain|
|     5|   10|   null|              null|    causes|     1|         |    causes|
|     6|   10|   null|              null|     flash|     1|         |     flash|
|     7|   10|   null|              null|  flooding|     1|         |  flooding|
|     8|   10|   null|              null|        of|     1|         |        of|
|     9|   10|   null|              null|   streets|     1|         |   streets|
|    10|   10|   null|      

In [12]:
#sentiment columns
#"_c0":"intensity","_c2": "word","_c3":"word_type", "_c4":"ny", "_c5":"posititvity"
sentiment_renamed = sentiment_with_schema.withColumnRenamed("_c0","intensity") \
                        .withColumnRenamed("_c1","numb") \
                        .withColumnRenamed("_c2","lword") \
                        .withColumnRenamed("_c3","word_type") \
                        .withColumnRenamed("_c4","ny") \
                        .withColumnRenamed("_c5","positivity") \
                        .withColumn(("lword"),lower("lword"))
sentiment_renamed = sentiment_renamed.drop(*['numb', 'ny','word_type'])
sentiment_renamed.show(3)

+---------+-----------+----------+
|intensity|      lword|positivity|
+---------+-----------+----------+
| weaksubj|  abandoned|  negative|
| weaksubj|abandonment|  negative|
| weaksubj|    abandon|  negative|
+---------+-----------+----------+
only showing top 3 rows



In [32]:
#create my own disaster table with an associated categorisation 
row_in=[("flood","natural"),
        ("fire","natural"),
        ("wave","natural"),
        ("extreme weather","natural"),
        ("tsunami","natural"),
        ("lightning","natural"),
        ("Earthquake","natural"),
        ("Apocalyptic","description"),
        ("disaster","description"),
        ("catastrophe","description"),
        ("tragedy","description"),
        ("doom","description"),
        ("death","description"),
        ("hell","description"),
        ("riot","human-related"),
        ("terrorism","human-related"),
        ("war","human-related"),
        ("warfare","human-related"),
        ("terror","description"),
        ("cataclysm","description"),
        ("agression","description"),
        ("killing","description"),
        ("burn","description"),
        ("burning","description"),
        ("suicide","description"),
        ("drawn","description"),
        ("armageddon","human-related"),
        ("hurricanes","natural"),
        ("tornados","natural"),
        ("cyclones","natural"),
        ("Wildfires","natural"),
        ("famine","natural"),
        ("droughts","natural"),
        ("landslide","natural"),
        ("wildfires","natural"), 
        ("ambulance","services"),
        ("emergency","services"),
        ("arrest","services"),
        ("police","services"),
        ("help","services"),
        ("bleeding","description"),
        ("slaughter","description"),
        ("dead","description"),
        ("fireman","services"),
        ("military","services"),
        ("assistance","services"),
        ("911","services")
       ]
schema = StructType([StructField('lword', StringType(),True),
                     StructField('disaster_type', StringType(),True)])
disaster_words = spark.createDataFrame(data = row_in,schema = schema).withColumn(("lword"),lower("lword"))

In [35]:
#merging sentiment and disaster tables
Combined = disaster_sample_expl.join(sentiment_renamed,"lword",how="left")
#removing duplicates
Combined = Combined.select("lword","mainID","id","keyword","location","intensity", "positivity","hashtag","target").distinct()
Combined.show(35)
Combined.select(count(when(col("hashtag") != "",True)).alias("Total_hashtag")).show() 



+--------------------+------+-----+-------+------------------+----------+----------+---------+------+
|               lword|mainID|   id|keyword|          location| intensity|positivity|  hashtag|target|
+--------------------+------+-----+-------+------------------+----------+----------+---------+------+
|               flood|     1|   10|   null|              null|      null|      null|   #flood|     1|
|            disaster|     2|   10|   null|              null|strongsubj|  negative|#disaster|     1|
|               heavy|     3|   10|   null|              null|      null|      null|         |     1|
|                rain|     4|   10|   null|              null|      null|      null|         |     1|
|              causes|     5|   10|   null|              null|      null|      null|         |     1|
|               flash|     6|   10|   null|              null|  weaksubj|  negative|         |     1|
|            flooding|     7|   10|   null|              null|      null|      nul

In [65]:
#join my disaster table - standardize the state of the tweets 
Combinedd = Combined.join(disaster_words,"lword", how='left')\
                    .withColumn(("location"),regexp_extract(col('location'), '.*,.*', 0))\
                    .orderBy("mainID")
Combinedd.show(100)

+--------------------+------+-----+-------+----------------+----------+----------+---------+------+-------------+
|               lword|mainID|   id|keyword|        location| intensity|positivity|  hashtag|target|disaster_type|
+--------------------+------+-----+-------+----------------+----------+----------+---------+------+-------------+
|               flood|     1|   10|   null|            null|      null|      null|   #flood|     1|      natural|
|            disaster|     2|   10|   null|            null|strongsubj|  negative|#disaster|     1|  description|
|               heavy|     3|   10|   null|            null|      null|      null|         |     1|         null|
|                rain|     4|   10|   null|            null|      null|      null|         |     1|         null|
|              causes|     5|   10|   null|            null|      null|      null|         |     1|         null|
|               flash|     6|   10|   null|            null|  weaksubj|  negative|      

In [37]:
#how many disaster words fit the dataset ?
Combinedd.select(count(when(col("disaster_type") != "",True)).alias("Total_disaster_word")).show() 

+-------------------+
|Total_disaster_word|
+-------------------+
|                951|
+-------------------+



In [38]:
#showing top words with at least 5 digits /avoiding operators/ and their respective sum
Combinedd.filter(length("lword") > 5) \
    .groupBy('lword') \
    .count().sort('count', ascending=False).show(20) 

+----------+-----+
|     lword|count|
+----------+-----+
|    people|  115|
| emergency|   90|
|  disaster|   71|
|   burning|   70|
|    police|   67|
|   suicide|   65|
|   content|   64|
|california|   63|
|   youtube|   58|
|   nuclear|   56|
|    severe|   56|
| hiroshima|   53|
|  families|   51|
| buildings|   50|
|    killed|   47|
|   another|   46|
|    attack|   45|
|    should|   43|
|    better|   42|
|   bombing|   42|
+----------+-----+
only showing top 20 rows



In [63]:
#top sentence with disaster words
Combinedd.filter(col("lword") != "") \
    .groupBy('mainID') \
    .count().sort('count', ascending=False).show(3) 

#number of sentence with disaster word
Combinedd.filter(col("disaster_type") != "") \
    .select(col("id")).distinct().count()

+------+-----+
|mainID|count|
+------+-----+
|  1829|    2|
| 63087|    2|
|  4935|    2|
+------+-----+
only showing top 3 rows



849

## Answering business questions

In [66]:
#who needs assisance
Combinedd.filter(col("disaster_type") == "services") \
    .select('*') \
    .show(30) 

+----------+------+-----+---------------+--------------------+---------+----------+-------+------+-------------+
|     lword|mainID|   id|        keyword|            location|intensity|positivity|hashtag|target|disaster_type|
+----------+------+-----+---------------+--------------------+---------+----------+-------+------+-------------+
|      help|   747|10116|       upheaval|                    | weaksubj|  positive|       |     0|     services|
|    police|  1321|10207|violent%20storm|                    |     null|      null|       |  null|     services|
|    police|  2095|10331|         weapon|        New York, NY|     null|      null|       |     1|     services|
|    police|  2326|10360|        weapons|                    |     null|      null|       |     0|     services|
|      help|  2470|10378|        weapons|                    | weaksubj|  positive|       |  null|     services|
|      help|  2494| 1038|       bleeding|     Vero Beach , FL| weaksubj|  positive|       |     

In [72]:
#Gravity of the situation 
Combinedd.filter((col("intensity") == "strongsubj") & (col("disaster_type") != "")) \
    .select('*') \
    .show(30) 

+-----------+------+-----+-----------------+--------------------+----------+----------+------------+------+-------------+
|      lword|mainID|   id|          keyword|            location| intensity|positivity|     hashtag|target|disaster_type|
+-----------+------+-----+-----------------+--------------------+----------+----------+------------+------+-------------+
|   disaster|     2|   10|             null|                null|strongsubj|  negative|   #disaster|     1|  description|
|apocalyptic|    29|10004|          tsunami|                    |strongsubj|  negative|            |     0|  description|
|   disaster|   397|10058|          typhoon|                null|strongsubj|  negative|            |     1|  description|
|   disaster|   403|10058|          typhoon|                null|strongsubj|  negative|            |     1|  description|
|   disaster|   446|10067|          typhoon|                    |strongsubj|  negative|            |     1|  description|
|   disaster|   452|1006

In [74]:
#states higher incident 
Combinedd.filter((col("lword") != "")  & (col("location") != "")) \
    .groupBy('location') \
    .count().sort('count', ascending=False).show(3) 

+---------------+-----+
|       location|count|
+---------------+-----+
|Los Angeles, CA|  264|
|    Chicago, IL|  182|
|California, USA|  159|
+---------------+-----+
only showing top 3 rows

